## ShellCommandTask

In addition to `FunctionTask`, pydra allows for creating tasks from shell commands by using `ShellCommandTask`.

Let's run a simple command `pwd` using pydra

In [ ]:
import pydra

In [ ]:
cmd = "pwd"
# we should use executable to pass the command we want to run
shelly = pydra.ShellCommandTask(name="shelly", executable=cmd)
shelly.cmdline

In [ ]:
# we can always check the cmdline of our task
shelly.cmdline

and now let's try to run it:

In [ ]:
with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)


and check the result

In [ ]:
shelly.result()

the result should have `return_code`, that hopehully is 0, `stdout` and `stderr`

#### longer commands
you can also use longer command by providing a list:

In [ ]:
cmd = ["echo", "hail", "pydra"]
shelly = pydra.ShellCommandTask(name="shelly", executable=cmd)
print("cmndline = ", shelly.cmdline)

with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)
shelly.result()

#### using args
In addition to `executable`, we can also use `args`. Last example can be also rewritten:

In [ ]:
cmd = "echo"
args = ["hail", "pydra"]

shelly = pydra.ShellCommandTask(name="shelly", executable=cmd, args=args)
print("cmndline = ", shelly.cmdline)

with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)
shelly.result()

#### customized input

Pydra always checks `executable` and `args`, but we can also provide additional inputs, in orger to do it, we have to modify `input_spec` first by using `SpecInfo` class:

In [ ]:
import attr

my_input_spec = pydra.specs.SpecInfo(
    name="Input",
    fields=[
        (
            "text",
            attr.ib(
                type=str,
                metadata={"position": 1, "help_string": "text", "mandatory": True},
                ),
        )
    ],
    bases=(pydra.specs.ShellSpec,),
)
# this will be simplified!

Now, we can define our task:

In [ ]:
cmd_exec = "echo"
hello = "HELLO"
shelly = pydra.ShellCommandTask(
    name="shelly", executable=cmd_exec, text=hello, input_spec=my_input_spec
)

print("cmndline = ", shelly.cmdline)

with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)
shelly.result()

#### customized output

We can also customized output if we want to return something more than the `stdout`, e.g. a file.

In [ ]:
my_output_spec = pydra.specs.SpecInfo(
    name="Output",
    fields=[("newfile", pydra.specs.File, "newfile_tmp.txt")],
    bases=(pydra.specs.ShellOutSpec,),
)


now we can create a task that returns a new file:

In [ ]:
cmd = ["touch", "newfile_tmp.txt"]
shelly = pydra.ShellCommandTask(name="shelly", executable=cmd, output_spec=my_output_spec)

print("cmndline = ", shelly.cmdline)

with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)
shelly.result()

#### exercise 1

Write a task that creates two new files, use provided output spec.

In [ ]:
cmd = "echo"
args = ["newfile_1.txt", "newfile_2.txt"]

my_output_spec = pydra.specs.SpecInfo(
    name="Output",
    fields=[
        (
            "out1",
            attr.ib(
                type=pydra.specs.File,
                metadata={
                    "output_file_template": "{args}",
                    "help_string": "output file",
                },
            ),
        )
    ],
    bases=(pydra.specs.ShellOutSpec,),
)


shelly = pydra.ShellCommandTask(name="shelly", executable=cmd, args=args, output_spec=my_output_spec).split("args")

print("cmndline = ", shelly.cmdline)

with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)
shelly.result()

In [ ]:
cmd = "echo"
args = ["newfile_1.txt", "newfile_2.txt"]

my_output_spec = pydra.specs.SpecInfo(
    name="Output",
    fields=[
        (
            "out1",
            attr.ib(
                type=pydra.specs.File,
                metadata={
                    "output_file_template": "{args}",
                    "help_string": "output file",
                },
            ),
        )
    ],
    bases=(pydra.specs.ShellOutSpec,),
)

# write your solution here

### DockerTask

all the commands can be also run in a docker container using `DockerTask`. Syntax is very similar, but additional argument `image` is required.

In [ ]:
cmd = "whoami"
docky = pydra.DockerTask(name="docky", executable=cmd, image="busybox")

with pydra.Submitter() as sub:
    docky(submitter=sub)

docky.result()

#### exercise2

Use splitter to run the same command in two different images:

In [ ]:
cmd = "whoami"
docky = pydra.DockerTask(name="docky", executable=cmd, image=["busybox", "ubuntu"]).split("image")

with pydra.Submitter() as sub:
    docky(submitter=sub)

docky.result()

In [ ]:
#write your solution here

#### Using `ShellCommandTask` with `container_info` argument:

You can run the shell command in a docker container by adding `container_info` argument to `ShellCommandTask`:

In [ ]:
shelly = pydra.ShellCommandTask(name="shelly", executable="whoami", container_info=("docker", "busybox"))
with pydra.Submitter() as sub:
    shelly(submitter=sub)

shelly.result()

If we don't provide `container_info` the output should be different:

In [ ]:
shelly = pydra.ShellCommandTask(name="shelly", executable="whoami")
with pydra.Submitter() as sub:
    shelly(submitter=sub)

shelly.result()